In [1]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import split, col
from pyspark.sql.types import TimestampType

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark.readStream\
      .format("socket")\
      .option("host", "localhost")\
      .option("port", 9999)\
      .load()\
        .writeStream\
      .format("console")\
        .outputMode("update")\
        .option("truncate", False) \
      .trigger(processingTime='10 seconds')\
      .start()\
      .awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------+
|2023-10-08 22:02:05.294|truck_geo_event|55|14|Paul Codding|5|Joplin to Kansas City|Normal|39.1|-94.59|1|                       |
|2023-10-08 22:02:06.124|truck_geo_event|76|30|Olivier Renault|11|Des Moines to Chicago Route 2|Normal|41.66|-90.82|1|          |
|2023-10-08 22:02:07.442|truck_geo_event|85|29|Mark Lochbihler|2|Wichita to Little Rock Route 2|Normal|36.24|-96.97|1|     

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [4]:
geoEventProcessing = spark.readStream\
                          .format("socket")\
                          .option("host", "localhost")\
                          .option("port", 9999)\
                          .load()\
                          .select(split("value",'\|').alias("fields"))\
                          .withColumn("timestamp_str",col("fields").getItem(0))\
                          .withColumn("timestamp",col("timestamp_str").cast(TimestampType()))\
                          .withColumn("driverName",col("fields").getItem(4))\
                          .withColumn("violation", col("fields").getItem(7))\
                          .where("violation!='Normal'")\
                          .groupBy("driverName", "violation")\
                          .count()\
                          .orderBy(col("count").desc())\
                          .limit(5)\
                          .writeStream\
                          .format("console")\
                          .outputMode("complete")\
                          .trigger(processingTime='10 seconds')\
                          .start()\
                          .awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+----------+---------+-----+
|driverName|violation|count|
+----------+---------+-----+
+----------+---------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|Suresh Srinivas|Unsafe following ...|    2|
|       Joe Witt|Unsafe tail distance|    1|
|       Joe Witt|           Overspeed|    1|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|Suresh Srinivas|Unsafe following ...|    2|
|       Joe Witt|      Lane Departure|    1|
|       Joe Witt|           Overspeed|    1|
|       Joe Witt|Unsafe tail distance|    1|
|Suresh Srinivas|      Lane Departure|    1|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|Suresh Srinivas|Unsafe following ...|    2|
|       Joe Witt|      Lane Departure|    2|
|       Joe Witt|Unsafe tail distance|    2|
|Suresh Srinivas|      Lane Departure|    2|
|Suresh Srinivas|Unsafe tail distance|    1|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|       Joe Witt|      Lane Departure|    4|
|Suresh Srinivas|Unsafe following ...|    2|
|Suresh Srinivas|Unsafe tail distance|    2|
|       Joe Witt|Unsafe tail distance|    2|
|Suresh Srinivas|      Lane Departure|    2|
+---------------+--------------------+-----+



...

-------------------------------------------
Batch: 7
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|       Joe Witt|      Lane Departure|    5|
|Suresh Srinivas|Unsafe following ...|    4|
|       Joe Witt|Unsafe tail distance|    4|
|Suresh Srinivas|Unsafe tail distance|    3|
|Suresh Srinivas|      Lane Departure|    3|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 8
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|       Joe Witt|      Lane Departure|    6|
|Suresh Srinivas|Unsafe following ...|    4|
|       Joe Witt|Unsafe tail distance|    4|
|Suresh Srinivas|      Lane Departure|    4|
|Suresh Srinivas|Unsafe tail distance|    3|
+---------------+--------------------+-----+



-------------------------------------------
Batch: 9
-------------------------------------------
+---------------+--------------------+-----+
|     driverName|           violation|count|
+---------------+--------------------+-----+
|       Joe Witt|      Lane Departure|    6|
|Suresh Srinivas|Unsafe tail distance|    5|
|Suresh Srinivas|Unsafe following ...|    4|
|       Joe Witt|Unsafe tail distance|    4|
|Suresh Srinivas|      Lane Departure|    4|
+---------------+--------------------+-----+



ERROR:root:KeyboardInterrupt while sending command.>            (155 + 2) / 200]
Traceback (most recent call last):
  File "/opt/spark3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 